<a href="https://colab.research.google.com/github/MMoronto/pytorch_exercises/blob/master/Peutingar_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Dictionary

In [1]:
pip install pytorch-forecasting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 KB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.2/147.2 KB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━

##Load data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

path1='/content/drive/MyDrive/Colab Notebooks/infra_data/Salient_cement_stats.xls'
path2='/content/drive/MyDrive/Colab Notebooks/infra_data/Salient_cement_stats.csv'
fob=open(path2,)
headings=next(fob)
for rec in fob:
  Cement3=rec.split(',')
  print(Cement3)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['"SALIENT CEMENT STATISTICS1', ' 2 "', '', '', '', '', '', '', '\n']
['', '', '', '', '', '', '', '\n']
['(Thousand metric tons unless otherwise specified)', '', '', '', '', '', '', '\n']
['', '', '', '', '', '', '', '\n']
['', '', '', '2014', '2015', '2016', '2017', '2018\n']
['United States:', '', '', '', '', '', '', '\n']
['Production:', '', '', '', '', '', '', '\n']
['Cement3', '', '', '"82', '535"', '"84', '405"', '"84', '695"', '"86', '356"', '"86', '400"\n']
['Clinker', '', '', '"74', '372"', '"76', '043"', '"75', '633"', '"76', '678"', '"77', '112"\n']
['"Shipments from mills and terminals:3', ' 4"', '', '', '', '', '', '', '\n']
['Quantity', '', '', '"88', '900"', '"92', '000"', '"94', '300"', '"96', '900"', '"98', '500"\n']
['Value5', 'thousand dollars', '', '"8', '940', '000"', '"9', '800', '000"', '"10', '500', '000"', '"11', '300', '000"', '"11'

In [3]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

os.chdir("../../..")

import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters


In [4]:
# Create a series and let pandas create a default integer index:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

In [5]:
from pandas.io.stata import DatetimeIndex
# Create a DataFrame by passing a NumPy array with a dateline index using `date_range()` & labeled columns:
dates = pd.date_range("20230101", periods=8)

dates

# DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04', '2013-01-05', '2013-01-06'], dtype='datatime64[ns]', freq='D')

df = pd.DataFrame(np.random.randn(8, 6), index=dates, columns=list("ABCDEF"))

df 

,A,B,C,D,E,F
2023-01-01,1.637430,-0.074899,-0.464288,2.592394,1.029729,0.367523
2023-01-02,0.554282,0.927489,0.615681,-0.056571,0.313035,0.928472
2023-01-03,1.567109,0.264060,0.732935,0.222612,0.838367,1.328711
2023-01-04,-0.963505,-1.245994,1.384940,-0.347263,1.337483,-0.767533
2023-01-05,-0.391329,0.873392,1.272581,1.373658,2.687006,-1.916265
2023-01-06,-0.286636,0.008211,-0.676744,-0.334472,1.042333,0.138345
2023-01-07,-1.295744,1.829233,-1.716455,-1.639073,0.097081,0.692098
2023-01-08,-0.536360,0.116404,0.038946,-0.543007,-0.521007,0.288568


In [6]:
# Create a `DataFrame` by passing a dictionary of objects that can be converted into a series-like structure:

df2 = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Timestamp("20130102"),
        "C": pd.Series(1, index=list(range(4)), dtype="float32"),
        "D": np.array([3] * 4, dtype="int32"),
        "E": pd.Categorical(["test", "train", "test", "train"]),
        "F": "foo",
    }
)

df2

,A,B,C,D,E,F
0,1.0,2013-01-02,1.0,3,test,foo
1,1.0,2013-01-02,1.0,3,train,foo
2,1.0,2013-01-02,1.0,3,test,foo
3,1.0,2013-01-02,1.0,3,train,foo


In [7]:
df2.dtypes

A           float64
B    datetime64[ns]
C           float32
D             int32
E          category
F            object
dtype: object

In [8]:
df2.D

0    3
1    3
2    3
3    3
Name: D, dtype: int32

##Viewing data

In [9]:
df.head()

,A,B,C,D,E,F
2023-01-01,1.637430,-0.074899,-0.464288,2.592394,1.029729,0.367523
2023-01-02,0.554282,0.927489,0.615681,-0.056571,0.313035,0.928472
2023-01-03,1.567109,0.264060,0.732935,0.222612,0.838367,1.328711
2023-01-04,-0.963505,-1.245994,1.384940,-0.347263,1.337483,-0.767533
2023-01-05,-0.391329,0.873392,1.272581,1.373658,2.687006,-1.916265


In [10]:
df.tail()

,A,B,C,D,E,F
2023-01-04,-0.963505,-1.245994,1.384940,-0.347263,1.337483,-0.767533
2023-01-05,-0.391329,0.873392,1.272581,1.373658,2.687006,-1.916265
2023-01-06,-0.286636,0.008211,-0.676744,-0.334472,1.042333,0.138345
2023-01-07,-1.295744,1.829233,-1.716455,-1.639073,0.097081,0.692098
2023-01-08,-0.536360,0.116404,0.038946,-0.543007,-0.521007,0.288568


In [11]:
df.columns

Index(['A', 'B', 'C', 'D', 'E', 'F'], dtype='object')

In [12]:
df.index

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08'],
              dtype='datetime64[ns]', freq='D')

`DataFrame.to_numpy()` gives a NumPy representation of the underlying data. 

In [13]:
df.to_numpy()

array([[ 1.6374296 , -0.07489908, -0.46428816,  2.59239403,  1.02972928,
         0.36752282],
       [ 0.55428243,  0.92748917,  0.61568067, -0.05657149,  0.31303534,
         0.92847216],
       [ 1.56710881,  0.26405989,  0.7329353 ,  0.22261165,  0.8383672 ,
         1.32871102],
       [-0.96350478, -1.2459937 ,  1.38494024, -0.34726348,  1.33748273,
        -0.7675332 ],
       [-0.39132864,  0.87339222,  1.27258135,  1.37365792,  2.68700573,
        -1.91626486],
       [-0.28663566,  0.00821121, -0.67674352, -0.33447245,  1.04233291,
         0.13834513],
       [-1.29574444,  1.82923336, -1.71645503, -1.63907349,  0.09708102,
         0.69209823],
       [-0.53635998,  0.11640397,  0.03894635, -0.54300727, -0.52100722,
         0.2885681 ]])

In [14]:
df2.to_numpy()

array([[1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'test', 'foo'],
       [1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'train', 'foo'],
       [1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'test', 'foo'],
       [1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'train', 'foo']],
      dtype=object)

In [15]:
df.describe()

,A,B,C,D,E,F
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,0.035656,0.337237,0.148450,0.158534,0.853003,0.132490
std,1.106109,0.902439,1.062259,1.291346,0.957412,1.032664
min,-1.295744,-1.245994,-1.716455,-1.639073,-0.521007,-1.916265
25%,-0.643146,-0.012566,-0.517402,-0.396199,0.259047,-0.088124
50%,-0.338982,0.190232,0.327314,-0.195522,0.934048,0.328045
75%,0.807489,0.886916,0.867847,0.510373,1.116120,0.751192
max,1.637430,1.829233,1.384940,2.592394,2.687006,1.328711


In [16]:
df

,A,B,C,D,E,F
2023-01-01,1.637430,-0.074899,-0.464288,2.592394,1.029729,0.367523
2023-01-02,0.554282,0.927489,0.615681,-0.056571,0.313035,0.928472
2023-01-03,1.567109,0.264060,0.732935,0.222612,0.838367,1.328711
2023-01-04,-0.963505,-1.245994,1.384940,-0.347263,1.337483,-0.767533
2023-01-05,-0.391329,0.873392,1.272581,1.373658,2.687006,-1.916265
2023-01-06,-0.286636,0.008211,-0.676744,-0.334472,1.042333,0.138345
2023-01-07,-1.295744,1.829233,-1.716455,-1.639073,0.097081,0.692098
2023-01-08,-0.536360,0.116404,0.038946,-0.543007,-0.521007,0.288568


In [17]:
df.T

,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08
A,1.637430,0.554282,1.567109,-0.963505,-0.391329,-0.286636,-1.295744,-0.536360
B,-0.074899,0.927489,0.264060,-1.245994,0.873392,0.008211,1.829233,0.116404
C,-0.464288,0.615681,0.732935,1.384940,1.272581,-0.676744,-1.716455,0.038946
D,2.592394,-0.056571,0.222612,-0.347263,1.373658,-0.334472,-1.639073,-0.543007
E,1.029729,0.313035,0.838367,1.337483,2.687006,1.042333,0.097081,-0.521007
F,0.367523,0.928472,1.328711,-0.767533,-1.916265,0.138345,0.692098,0.288568


In [18]:
df.sort_index(axis=1, ascending=False)

,F,E,D,C,B,A
2023-01-01,0.367523,1.029729,2.592394,-0.464288,-0.074899,1.637430
2023-01-02,0.928472,0.313035,-0.056571,0.615681,0.927489,0.554282
2023-01-03,1.328711,0.838367,0.222612,0.732935,0.264060,1.567109
2023-01-04,-0.767533,1.337483,-0.347263,1.384940,-1.245994,-0.963505
2023-01-05,-1.916265,2.687006,1.373658,1.272581,0.873392,-0.391329
2023-01-06,0.138345,1.042333,-0.334472,-0.676744,0.008211,-0.286636
2023-01-07,0.692098,0.097081,-1.639073,-1.716455,1.829233,-1.295744
2023-01-08,0.288568,-0.521007,-0.543007,0.038946,0.116404,-0.536360


In [19]:
df.sort_values(by="B")

,A,B,C,D,E,F
2023-01-04,-0.963505,-1.245994,1.384940,-0.347263,1.337483,-0.767533
2023-01-01,1.637430,-0.074899,-0.464288,2.592394,1.029729,0.367523
2023-01-06,-0.286636,0.008211,-0.676744,-0.334472,1.042333,0.138345
2023-01-08,-0.536360,0.116404,0.038946,-0.543007,-0.521007,0.288568
2023-01-03,1.567109,0.264060,0.732935,0.222612,0.838367,1.328711
2023-01-05,-0.391329,0.873392,1.272581,1.373658,2.687006,-1.916265
2023-01-02,0.554282,0.927489,0.615681,-0.056571,0.313035,0.928472
2023-01-07,-1.295744,1.829233,-1.716455,-1.639073,0.097081,0.692098


##Getting

In [20]:
# Select a single column, which yields a Series, equivalent to 'df.A'
df["A"]

2023-01-01    1.637430
2023-01-02    0.554282
2023-01-03    1.567109
2023-01-04   -0.963505
2023-01-05   -0.391329
2023-01-06   -0.286636
2023-01-07   -1.295744
2023-01-08   -0.536360
Freq: D, Name: A, dtype: float64

In [21]:
df[0:3]

,A,B,C,D,E,F
2023-01-01,1.637430,-0.074899,-0.464288,2.592394,1.029729,0.367523
2023-01-02,0.554282,0.927489,0.615681,-0.056571,0.313035,0.928472
2023-01-03,1.567109,0.264060,0.732935,0.222612,0.838367,1.328711


##Selection by label

In [22]:
df.loc[dates[1]]

A    0.554282
B    0.927489
C    0.615681
D   -0.056571
E    0.313035
F    0.928472
Name: 2023-01-02 00:00:00, dtype: float64

In [23]:
#Selecting on a multi-axis by label:
df.loc[:, ["A", "D"]]

,A,D
2023-01-01,1.637430,2.592394
2023-01-02,0.554282,-0.056571
2023-01-03,1.567109,0.222612
2023-01-04,-0.963505,-0.347263
2023-01-05,-0.391329,1.373658
2023-01-06,-0.286636,-0.334472
2023-01-07,-1.295744,-1.639073
2023-01-08,-0.536360,-0.543007


In [24]:
# Showing label slicing, both endpoints are included:
df.loc["20230104":"20230106", ["A", "D"]]

,A,D
2023-01-04,-0.963505,-0.347263
2023-01-05,-0.391329,1.373658
2023-01-06,-0.286636,-0.334472


In [25]:
# Reduction in the dimensions of the returned object:
df.loc["20230104", ["A", "D"]]

A   -0.963505
D   -0.347263
Name: 2023-01-04 00:00:00, dtype: float64

In [26]:
# For getting a scalar value
df.loc[dates[1], "D"]

-0.05657148741595979

In [27]:
# For getting fast access to a scalar value
df.at[dates[1], "D"]

-0.05657148741595979

##Selection by position

In [28]:
df.iloc[3]

A   -0.963505
B   -1.245994
C    1.384940
D   -0.347263
E    1.337483
F   -0.767533
Name: 2023-01-04 00:00:00, dtype: float64

In [29]:
df.iloc[3:5, 0:2]

,A,B
2023-01-04,-0.963505,-1.245994
2023-01-05,-0.391329,0.873392


In [30]:
df.iloc[[1, 2, 4], [0, 2]]

,A,C
2023-01-02,0.554282,0.615681
2023-01-03,1.567109,0.732935
2023-01-05,-0.391329,1.272581


In [31]:
# For slicing rows explicitly
df.iloc[1:3, :]

,A,B,C,D,E,F
2023-01-02,0.554282,0.927489,0.615681,-0.056571,0.313035,0.928472
2023-01-03,1.567109,0.264060,0.732935,0.222612,0.838367,1.328711


In [34]:
# For slicing columns explicitly
df.iloc[:, 1:3]

,B,C
2023-01-01,-0.074899,-0.464288
2023-01-02,0.927489,0.615681
2023-01-03,0.264060,0.732935
2023-01-04,-1.245994,1.384940
2023-01-05,0.873392,1.272581
2023-01-06,0.008211,-0.676744
2023-01-07,1.829233,-1.716455
2023-01-08,0.116404,0.038946


In [35]:
# For getting values explicitly
df.iloc[1, 1]

0.9274891736235603

In [36]:
# For getting fast access to scalar (equivalent to the prior method)
df.iat[1, 1]

0.9274891736235603

## Boolean indexing
Using a single column's values to select data:

In [37]:
df[df["A"] > 0]

,A,B,C,D,E,F
2023-01-01,1.637430,-0.074899,-0.464288,2.592394,1.029729,0.367523
2023-01-02,0.554282,0.927489,0.615681,-0.056571,0.313035,0.928472
2023-01-03,1.567109,0.264060,0.732935,0.222612,0.838367,1.328711


In [38]:
# Selecting values from a DataFrame where a boolean condition is met:
df[df > 0]

,A,B,C,D,E,F
2023-01-01,1.637430,NaN,NaN,2.592394,1.029729,0.367523
2023-01-02,0.554282,0.927489,0.615681,NaN,0.313035,0.928472
2023-01-03,1.567109,0.264060,0.732935,0.222612,0.838367,1.328711
2023-01-04,NaN,NaN,1.384940,NaN,1.337483,NaN
2023-01-05,NaN,0.873392,1.272581,1.373658,2.687006,NaN
2023-01-06,NaN,0.008211,NaN,NaN,1.042333,0.138345
2023-01-07,NaN,1.829233,NaN,NaN,0.097081,0.692098
2023-01-08,NaN,0.116404,0.038946,NaN,NaN,0.288568
